## Introduction

In this project I will explore the culinaric possibilities in Cologne. It is a city which has a diverse offering of different Bars Restaurants and Cafes and it is nearly impossible to get to know all of them, not mentioning that there are closing and opening up new venues from time to time. As a person living in or visiting this city, there is so much to explore and a goal of this project is to help identifying those venues that are worth visiting. 
As a way of achieving this, I tried to model a basic scoring system, that takes into account the rating, the price and the distance of the venue depending on the current location. It then recommends you a list of venues with the best scoring, independent of the actual category of the venue. Therefore this method is not really suitable for all user, for instance those who look for a specific kind of venue or cuisine. However, for users that are new to the city it provides venues that are objectively worth visiting.

## Data
The available FourSquare location Data via the Foursquare API provide us with the necessary information to visualize the restaurants that are spread throughout the city. On a more detailed point of view, we can dive into specific neigbourhood or streets and find the best rated venues, the distance to the venue and the respective price category.
All postal codes from the city of Cologne are available on "https://www.koeln.de/postleitzahlen" and the necessary location from the respective Cologne postal codes can be found on "http://www.fa-technik.adfc.de/code/opengeodb/PLZ.tab", which is a csv-file, similar to the Toronto example we had troughout the course.

## Methodology
Postal code data will be collected from https://www.koeln.de/postleitzahlen, cleaned and processed into a dataframe.
The respective location data is read into a dataframe from the mentioned csv-file and merged with the postal code data including the neigbourhood names.
With the available location data we will use our Foursquare Developer account to fetch the detailed venue information such as category, price, rating etc.
Then we create a score with the available venue information to output those venues that are ranked highest with our applied scoring system. These are the venues that are reccommended to visit.

## Problem Statement
What are the objectively best venues to visit in the current area?

To see that, let's start bei installing and importing all necessary libraries

In [1]:
pip install folium

     |████████████████████████████████| 93 kB 4.5 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests

from bs4 import BeautifulSoup
import os

import folium # map rendering library

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

import seaborn as sns

print('Libraries imported.')

Libraries imported.


Now define the necessary functions

In [3]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


In [4]:
location = geo_location("Cologne")
latitude, longitude = location

In [5]:
url = requests.get('https://www.koeln.de/postleitzahlen')
url.content
soup = BeautifulSoup(url.content, "lxml")
list_plz = []
for plz_html in soup.find_all("ul", attrs={"class":"plz"}):
    for plz in plz_html.find_all('li'):
        list_plz.append(plz.text)

df = pd.DataFrame(list_plz, columns=["plz"])
df.dtypes

plz    object
dtype: object

In [6]:
try:
    from urllib.request import Request, urlopen  # Python 3
except ImportError:
    from urllib2 import Request, urlopen  # Python 2

df_geo = pd.read_csv("http://www.fa-technik.adfc.de/code/opengeodb/PLZ.tab", sep='\t',  dtype=str)
df_geo

df = df.merge(df_geo, on ='plz', how='left')
df

,plz,#loc_id,lon,lat,Ort
0,50667,8321,6.95768491832349,50.9400886235176,Köln
1,50668,8322,6.96545005139455,50.950695435874,Köln
2,50670,8323,6.95095743959049,50.9511722093173,Köln
3,50672,8309,6.9384761893034,50.9434409895397,Köln
4,50674,8324,6.93704081933882,50.933583841904,Köln
5,50676,8325,6.95555962327279,50.9329782103951,Köln
6,50677,8326,6.95323649210595,50.9225551674677,Köln
7,50678,8327,6.96648245395115,50.9240361563136,Köln
8,50679,8328,6.98103226157441,50.9374228290525,Köln
9,50733,8329,6.95639738632809,50.9651646910743,Köln


In [7]:
# create map of Cologne using latitude and longitude values
map_cologne = folium.Map(location=location, zoom_start=12)

# add markers to map
for lat, lng, Ort, plz in zip(df['lat'], df['lon'], df['Ort'], df['plz']):
    label = '{}, {}'.format(Ort, plz)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cologne)  
    
map_cologne

Now we sign into our FourSquare Developer account

In [8]:
CLIENT_ID = 'OSEO0CTWLPBA5D4C3SUQ55EE1JLPPPAIYN3RYAECQUI10L3U' # your Foursquare ID
CLIENT_SECRET = 'ENUQSRXBH00V30AEHILA3QP5HLCXHRDH2HNVQFMG5HIAGHGX' # your Foursquare Secret
ACCESS_TOKEN = 'UNOFVKGGOI1JNSOCH43MV2LQHGFGUG4CBTED51J53A4M0UEA' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OSEO0CTWLPBA5D4C3SUQ55EE1JLPPPAIYN3RYAECQUI10L3U
CLIENT_SECRET:ENUQSRXBH00V30AEHILA3QP5HLCXHRDH2HNVQFMG5HIAGHGX


Filter those results on category ID "4d4b7105d754a06374d81259", which is Food Places, and set the limit to 750 meters.

In [9]:
category = '4d4b7105d754a06374d81259'
radius = 750

Define a function that return the location data of a specific area code.

In [10]:
#create function to get location of plz
def get_location(area_code):
    # get geo location of area code
    s= df[df['plz'] == area_code]
    latitude_plz = s['lat'].values[0]
    longitude_plz = s['lon'].values[0]
    return latitude_plz, longitude_plz

For my example we seek for the longitude and latitude of the postal code "50674", which is the area I am living in.

In [11]:
latitude, longitude = get_location('50674')

Then create the url to request the venue information

In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit{}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, category, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OSEO0CTWLPBA5D4C3SUQ55EE1JLPPPAIYN3RYAECQUI10L3U&client_secret=ENUQSRXBH00V30AEHILA3QP5HLCXHRDH2HNVQFMG5HIAGHGX&ll=50.933583841904,6.93704081933882&oauth_token=UNOFVKGGOI1JNSOCH43MV2LQHGFGUG4CBTED51J53A4M0UEA&v=20180604&categoryId=4d4b7105d754a06374d81259&radius=750&limit200'

Define some functions to 1) get a list a venue depending on the location point and 2) get the detailed information of the venueID.

In [37]:
def get_venues(latitude,longitude):
    #set variables
    radius=750
    LIMIT=100
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            category)
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_distance=row['venue']['location']['distance']
            venue_plz=row['venue']['location']['postalCode']
            venue_address=row['venue']['location']['address']
            venue_details.append([venue_id,venue_name,venue_category, venue_distance, venue_plz, venue_address])
        except KeyError:
            pass
    column_names=['ID','Name','Category', 'Distance', 'PostalCode', 'Address']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    #get all the data
    results = requests.get(url).json()
    #print(results)
    venue_data=results['response']['venue']
    print(venue_data)
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_price=venue_data['price']['tier']
        #venue_price=venue_data['listed']['groups']['attributes']['groups']['items']['priceTier']
        venue_rating=venue_data['rating']
        print(venue_price)
        venue_details.append([venue_id,venue_name,venue_price,venue_rating])
        print(venue_details)
    except KeyError:
        pass
    column_names=['ID','Name','Price','Rating']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


In [38]:
get_venue_details('4b27e448f964a520cc8b24e3')

{'id': '4b27e448f964a520cc8b24e3', 'name': 'Brauhaus A. Pütz', 'contact': {'phone': '+49221211166', 'formattedPhone': '+49 221 211166'}, 'location': {'address': 'Engelbertstr. 67', 'lat': 50.93512, 'lng': 6.9377885, 'labeledLatLngs': [{'label': 'display', 'lat': 50.93512, 'lng': 6.9377885}], 'postalCode': '50674', 'cc': 'DE', 'city': 'Köln', 'state': 'Nordrhein-Westfalen', 'country': 'Deutschland', 'formattedAddress': ['Engelbertstr. 67', '50674 Köln', 'Deutschland']}, 'canonicalUrl': 'https://foursquare.com/v/brauhaus-a-p%C3%BCtz/4b27e448f964a520cc8b24e3', 'categories': [{'id': '50327c8591d4c4b30a586d5d', 'name': 'Brewery', 'pluralName': 'Breweries', 'shortName': 'Brewery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 21}, 'price': {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 'likes': {'count': 102, 'groups': [{'type': 'others', 'count': 102, 'items': []}], 'summary': '

,ID,Name,Price,Rating
0,4b27e448f964a520cc8b24e3,Brauhaus A. Pütz,2,8.3


In [39]:
plz_venues = get_venues(latitude,longitude).sort_values(by=['Distance'])
plz_venues

,ID,Name,Category,Distance,PostalCode,Address
5,5c084427061b51002c127368,Tapeo & Co.,Tapas Restaurant,81,50674,Lindenstr. 38
3,57645427498e7a5746a35766,Tanica,Italian Restaurant,142,50674,Engelbertstraße 31
80,5a3a15691f8ed64718884d30,Tigermilch,Peruvian Restaurant,176,50674,Brüsseler Str. 12
0,583ec82c19b1ad33a488ae04,mikoto,Sushi Restaurant,196,59674,Hohenstaufenring 55
2,4b05886bf964a52082c422e3,Café Wahlen,Café,206,50674,Hohenstaufenring 64
...,...,...,...,...,...,...
82,4ee268668231cd14a32afac3,Shaka Zulu,African Restaurant,716,50672,Limburger Str. 29
85,5159889ee4b0655021dd3bd5,Hot Point,Chinese Restaurant,719,50667,Salierring 44
83,4fb50c37e4b0186ba39cc480,Caveedel,Café,724,50672,Brüsseler Str. 69
94,4d91eae49acaa143a55af2f0,Lakshmi,Indian Restaurant,738,50676,Thieboldsgasse 101-103


In [40]:
dfObj = pd.DataFrame(columns=['ID' 'Price', 'Rating'])
for column in plz_venues['ID']:
        try:
            detail=get_venue_details(column)
            dfObj = pd.concat([dfObj, detail])
        except KeyError:
            pass


{'id': '5c084427061b51002c127368', 'name': 'Tapeo & Co.', 'contact': {'phone': '+4922182082000', 'formattedPhone': '+49 221 82082000'}, 'location': {'address': 'Lindenstr. 38', 'crossStreet': 'Jean-Claude-Letist-Platz', 'lat': 50.93382, 'lng': 6.935945, 'labeledLatLngs': [{'label': 'display', 'lat': 50.93382, 'lng': 6.935945}], 'postalCode': '50674', 'cc': 'DE', 'city': 'Köln', 'state': 'Nordrhein-Westfalen', 'country': 'Deutschland', 'formattedAddress': ['Lindenstr. 38 (Jean-Claude-Letist-Platz)', '50674 Köln', 'Deutschland']}, 'canonicalUrl': 'https://foursquare.com/v/tapeo--co/5c084427061b51002c127368', 'categories': [{'id': '4bf58dd8d48988d1db931735', 'name': 'Tapas Restaurant', 'pluralName': 'Tapas Restaurants', 'shortName': 'Tapas', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tapas_', 'suffix': '.png'}, 'primary': True}, {'id': '4bf58dd8d48988d11e941735', 'name': 'Cocktail Bar', 'pluralName': 'Cocktail Bars', 'shortName': 'Cocktail', 'icon': {'prefix': 'https:

Now we merge the venue list with the respective venue details.

In [41]:
#dfObj
#plz_venues
score_venues = pd.merge(plz_venues, dfObj, on='ID')
score_venues


,ID,Name_x,Category,Distance,PostalCode,Address,Name_y,Price,Rating
0,5c084427061b51002c127368,Tapeo & Co.,Tapas Restaurant,81,50674,Lindenstr. 38,Tapeo & Co.,1,8.0
1,57645427498e7a5746a35766,Tanica,Italian Restaurant,142,50674,Engelbertstraße 31,Tanica,2,8.1
2,5a3a15691f8ed64718884d30,Tigermilch,Peruvian Restaurant,176,50674,Brüsseler Str. 12,Tigermilch,1,7.0
3,583ec82c19b1ad33a488ae04,mikoto,Sushi Restaurant,196,59674,Hohenstaufenring 55,mikoto,1,8.6
4,4b05886bf964a52082c422e3,Café Wahlen,Café,206,50674,Hohenstaufenring 64,Café Wahlen,2,8.2
5,5b1d6efaa30619002c440c7d,vevi - veganes vintage café,Vegetarian / Vegan Restaurant,254,50674,Brüsseler Str 29,vevi - veganes vintage café,2,8.3
6,4d94c3833048b1f73eed3864,Gate to India,Indian Restaurant,274,50674,Engelbertstr. 17,Gate to India,2,7.1
7,52861309498eb1a0d3bb4b45,Bonjour Saigon,Vietnamese Restaurant,280,50674,Richard-Wagner-Str. 24,Bonjour Saigon,2,7.7
8,4fdf9403e4b0a1e6c282807d,Alborz,Middle Eastern Restaurant,280,50674,Rathenauplatz 1,Alborz,2,8.3
9,4b31415ef964a520590325e3,Fischermanns',Restaurant,285,50674,Rathenauplatz 21,Fischermanns',1,7.6


In [53]:
score_venues2 = score_venues.astype({"Price": int})

In [54]:
max_dist = score_venues2["Distance"].max()
max_rating = score_venues2["Rating"].max()
max_price = score_venues2["Price"].max()
max_price

2

Create a new column containing the calculated score

In [78]:
score_venues2["Score"] = (1-(score_venues2["Distance"]/max_dist))*0.15 + score_venues2["Rating"]/max_rating*0.7 + (1-score_venues2["Price"]/max_price)*0.15
score_venues2.sort_values(by=['Score'], ascending=False)

,ID,Name_x,Category,Distance,PostalCode,Address,Name_y,Price,Rating,Score
3,583ec82c19b1ad33a488ae04,mikoto,Sushi Restaurant,196,59674,Hohenstaufenring 55,mikoto,1,8.6,0.825833
0,5c084427061b51002c127368,Tapeo & Co.,Tapas Restaurant,81,50674,Lindenstr. 38,Tapeo & Co.,1,8.0,0.819097
26,4ba20b35f964a52001d837e3,Meister Gerhard no 008,Tapas Restaurant,357,50674,Rathenauplatz 8,Meister Gerhard no 008,1,8.8,0.785486
15,4b05886af964a52045c422e3,Orlando,Café,316,50764,Engelbertstr. 9,Orlando,1,8.6,0.784167
10,54d7540d498ed907b5f12ed4,Bangkok,Thai Restaurant,285,50674,Lindenstraße 81,Bangkok,1,8.2,0.763819
12,4b05886ff964a520e9c522e3,El Inca,Peruvian Restaurant,294,50674,Görresstr. 2,El Inca,1,8.2,0.760694
13,51d506fe498e56324b31353c,Gernys Schnelleinkauf,Soup Place,296,50676,Schaafenstr. 53-55,Gernys Schnelleinkauf,1,7.9,0.736667
29,4b0e556ff964a520e35623e3,cafecafe,Café,401,50674,Aachener Str. 45,cafecafe,1,8.3,0.731319
1,57645427498e7a5746a35766,Tanica,Italian Restaurant,142,50674,Engelbertstraße 31,Tanica,2,8.1,0.730694
19,4b058869f964a5202ec422e3,Café Feynsinn,Café,329,50674,Rathenauplatz 7,Café Feynsinn,1,7.8,0.717431


In [72]:
(1-score_venues2["Rating"]/max_rating)

0     0.111111
1     0.100000
2     0.222222
3     0.044444
4     0.088889
5     0.077778
6     0.211111
7     0.144444
8     0.077778
9     0.155556
10    0.088889
11    0.233333
12    0.088889
13    0.122222
14    0.144444
15    0.044444
16    0.066667
17    0.055556
18    0.077778
19    0.133333
20    0.122222
21    0.088889
22    0.077778
23    0.166667
24    0.177778
25    0.055556
26    0.022222
27    0.111111
28    0.211111
29    0.077778
30    0.177778
31    0.000000
32    0.044444
33    0.055556
Name: Rating, dtype: float64

In [19]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_food.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df_food.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
#filter only category restaurant


# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.drop(columns = ["labeledLatLngs", "formattedAddress"], inplace = True)
dataframe_filtered[dataframe_filtered["categories"].str.contains("Restaurant", na = False)]
dataframe_filtered

NameError: name 'df_food' is not defined

In [ ]:
cologne_onehot = pd.get_dummies(plz_venues[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cologne_onehot['PostalCode'] = plz_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [cologne_onehot.columns[-1]] + list(cologne_onehot.columns[:-1])
cologne_onehot = cologne_onehot[fixed_columns]
cologne_grouped = cologne_onehot.groupby('PostalCode').mean().reset_index()
cologne_grouped.head()

In [ ]:
import matplotlib.pyplot as plotter
figureObject, axesObject = plotter.subplots()


# Draw the pie chart
axesObject.pie(cologne_grouped,
        autopct='%1.2f',
        startangle=90)
# Aspect ratio - equal means pie is a circle
axesObject.axis('equal')

plotter.show()

In [ ]:
# create map of Cologne using latitude and longitude values
map_food = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, name, postalCode in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['postalCode']):
    label = '{}, {}'.format(name, postalCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_food)  
    
map_food